# Import Libraries 

In [2]:
import findspark                                      #Import library to Search for Spark Installation  

findspark.init()                                      #Search Spark Installation

import pandas as pd                                   #Import Pandas
import requests
from pandas.io.json import json_normalize             #Import JSON normalizer 
import json

from pyspark.sql import SparkSession                  # Import of Spark Session
from pyspark import SparkContext as spark             # Import the Regular Spark Contex
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()
import os

import pyarrow.parquet as pq                          #Create Parquet File

sc = spark.sparkContext                               #Initialize Spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1576061099095_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------

# Creating Variables

## Buckets

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

## Output Paths

In [ ]:
SPG_BUCKET_IBGE_OUTPUT_PATH = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_IBGE/SPG_RG_IBGE.parquet"

## Creating Global Variables

In [3]:
state_list_text = []       #Array with States as Text
state_list_json = []       #Array with States as JSON 
cities_list_text = []      #Array with Cities as Text
cities_list_json = []      #Array with Cities as JSON
df_cities = pd.DataFrame() #Dataframe with Cities

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
states_URL = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
cities_main_URL = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/"
cities_URL_complement = "/municipios"

headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
          }           #Header Configuration

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Call State API And Transform to Pandas DF

In [5]:
state_list_text = requests.get(states_URL, headers=headers).text  #Getting the total number of states and there IDs as Text
state_list_json = json.loads(state_list_text)                        #Converting the "state_list_text" as JSON
states_normalized = json_normalize(state_list_json)                  #Normalizing the JSON
df_state = pd.DataFrame(states_normalized)                           #Creating a Pandas DF with the Normalized JSON

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# For Loop that Call the Cities API, Load, Normalize and Insert into a DataFrame

In [6]:
for i, row in df_state.iterrows():                                                          #Getting all the cities per states
    cities_list_text = requests.get(cities_main_URL
                                    + str(df_state.id[i])
                                    + cities_URL_complement
                                    , headers=headers).text                                 # Call the city API
    cities_list_json = json.loads(cities_list_text)                                         # Load the cities_list_text as JSON                  
    cities_normalized = json_normalize(cities_list_json)                                    # Normalize the JSON as Table 
    df_cities = df_cities.append(cities_normalized, ignore_index=True)                      # Append to df_cities DataFrame

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Change the Columns Name

In [7]:
df_cities = df_cities.rename(index=str, columns={"id": "IBGE_ID"
                                                 ,"nome":"IBGE_NAME"
                                                 ,"microrregiao.id": "IBGE_MICRO_ID"
                                                 ,"microrregiao.nome":"IBGE_MICRO_NAME"
                                                 ,"microrregiao.mesorregiao.UF.id": "IBGE_UF_ID"
                                                 ,"microrregiao.mesorregiao.UF.nome": "IBGE_UF_NAME"
                                                 ,"microrregiao.mesorregiao.UF.regiao.id": "IBGE_REGION_ID"
                                                 ,"microrregiao.mesorregiao.UF.regiao.nome": "IBGE_REGION_NAME"
                                                 ,"microrregiao.mesorregiao.UF.regiao.sigla": "IBGE_REGION_ACRONYMS"
                                                 ,"microrregiao.mesorregiao.UF.sigla":"IBGE_UF_ACRONYMS"
                                                 ,"microrregiao.mesorregiao.id":"IBGE_MESO_ID"
                                                 ,"microrregiao.mesorregiao.nome":"IBGE_MESO_NAME"})                 #Chaging the name of the columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark_df_IBGE_cities = spark.createDataFrame(df_cities)                                   #Converting the df_cities (Pandas Df) to a Spark DF
spark_df_IBGE_cities = spark_df_IBGE_cities.repartition(144)                  #Repartitioning the Spark DF

In [ ]:
spark_df_IBGE_cities.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_BUCKET_IBGE_OUTPUT_PATH, mode = "overwrite")